## Voronoi expansion implementation in DEVS Simulation Framework 


### Attemping to make an animated matplotlib celldevs plot
Using as example project a Vornoi Zones implementation in cellDevs.

See this links.
- http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/
- https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.load.html
- https://www.numpy.org/devdocs/reference/generated/numpy.lib.format.html
- https://github.com/rogersce/cnpy

In [33]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10,8)
import matplotlib.animation as animation
from IPython.display import HTML

In [34]:
%load_ext autoreload
%autoreload 2
from pycdevs.cdppwrapper import CDPPWrapper, DrawlogFailedException
from pycdevs.models import Model, CellValues
from pycdevs import cdppwrapper as cdppExceptions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import random

inital_cell_values = np.zeros((200, 200), dtype=int)
get_random_i = lambda : random.randint(0, inital_cell_values.shape[0])
get_random_j = lambda : random.randint(0, inital_cell_values.shape[1])
current_value = 1

for _ in range(5):
    inital_cell_values[get_random_i(), get_random_j()] = current_value
    print(f'Random value asigned')
    current_value += 1
    
devs_initiall_cell_values = CellValues(inital_cell_values, 0)

Random value asigned
Random value asigned
Random value asigned
Random value asigned
Random value asigned


In [48]:
modelSourceCodeHeader = """
[Top]
components : voronoiExpansion

[voronoiExpansion]
type : cell
dim : ${cellValuesDimension}
delay : transport
defaultDelayTime  : 0
border : nowrapped
neighbors : ${neighboursList}
initialvalue : 0
initialCellsValue : ${cellValuesPath}
localtransition : voronoiSite

[voronoiSite]
"""
# TODO: Make some kind of meta-DSL for replacements and formulaes inside CellDEVS model DSL
modelSourceCodeHeader = modelSourceCodeHeader\
    .replace('${cellValuesDimension}', devs_initiall_cell_values.get_dimension())\
    .replace('${cellValuesPath}', devs_initiall_cell_values.write_to_file())

Programatically defining **diamond shaped voronoi**

In [49]:
def getNeighbourRule(aNeighbour):
    return "rule : {%s} ${expansionWaveDelay} { (0,0)=0 and %s!=0 and %s!=? }" % (aNeighbour, aNeighbour, aNeighbour)

def getDefaultRule():
    return "rule : {(0,0)} 0 { t }"

# Diamond shaped neighbourhood
neighbours = [
    (0, -2), \
    (-1,-1), (0, -1), (1,-1), \
    (-2, 0), (-1,0), (1,0), (2, 0),\
    (-1,1), (0, 1), (1,1), \
    (0, 2)
]
rules ="""
% Expand center cell value in a 3x3 square neighbourhood
"""
for neighbour in neighbours:
    rules = rules + getNeighbourRule(str(neighbour).replace(' ', '')) + '\n'
rules = rules + getDefaultRule() + '\n'
rules = rules.replace('${expansionWaveDelay}', str(100))

In [50]:
modelSourceCodeHeader = modelSourceCodeHeader.replace('${neighboursList}', \
    " ".join(['voronoiExpansion'+str(neighbour).replace(' ', '') for neighbour in neighbours + [(0,0)]]) \
)

In [51]:
modelSourceCode = modelSourceCodeHeader + rules
simpleVoronoiModel = Model(modelSourceCode, 'voronoiExpansion')

This is how the model source code ends up being generated.

In [52]:
print(modelSourceCode)


[Top]
components : voronoiExpansion

[voronoiExpansion]
type : cell
dim : (200,200)
delay : transport
defaultDelayTime  : 0
border : nowrapped
neighbors : voronoiExpansion(0,-2) voronoiExpansion(-1,-1) voronoiExpansion(0,-1) voronoiExpansion(1,-1) voronoiExpansion(-2,0) voronoiExpansion(-1,0) voronoiExpansion(1,0) voronoiExpansion(2,0) voronoiExpansion(-1,1) voronoiExpansion(0,1) voronoiExpansion(1,1) voronoiExpansion(0,2) voronoiExpansion(0,0)
initialvalue : 0
initialCellsValue : /tmp/1543707589368.val
localtransition : voronoiSite

[voronoiSite]

% Expand center cell value in a 3x3 square neighbourhood
rule : {(0,-2)} 100 { (0,0)=0 and (0,-2)!=0 and (0,-2)!=? }
rule : {(-1,-1)} 100 { (0,0)=0 and (-1,-1)!=0 and (-1,-1)!=? }
rule : {(0,-1)} 100 { (0,0)=0 and (0,-1)!=0 and (0,-1)!=? }
rule : {(1,-1)} 100 { (0,0)=0 and (1,-1)!=0 and (1,-1)!=? }
rule : {(-2,0)} 100 { (0,0)=0 and (-2,0)!=0 and (-2,0)!=? }
rule : {(-1,0)} 100 { (0,0)=0 and (-1,0)!=0 and (-1,0)!=? }
rule : {(1,0)} 100 { (0,

In [53]:
# TODO: VTime builder wrapper
wrapper = CDPPWrapper(simpleVoronoiModel, '00:01:00:00')
print(" ".join(wrapper.getArguments()))

cd++ -m/tmp/1543707601040.ma -o/tmp/1543707601040.out -t00:01:00:00 -l/tmp/1543707601040.log


In [55]:
try:
    wrapper.run()
except cdppExceptions.SimulationExecutedButFailedException:
    print(wrapper.getSimulationOutput())

In [56]:
try:
    drawlogNPFilename = wrapper.drawlog('00:00:00:100')
except DrawlogFailedException as e:
    print(e.stderr.decode('ascii'))

In [57]:
loadedMatrix = np.load(drawlogNPFilename)
# loadedMatrix = loadedMatrix.reshape(cellDevsShape)

composedMatrix = np.zeros([devs_initiall_cell_values.values.shape[0], devs_initiall_cell_values.values.shape[1], len(loadedMatrix.files)], dtype=np.double)

for index, npFileName in enumerate(loadedMatrix.files):
    composedMatrix[..., index] = loadedMatrix[ npFileName ].reshape(devs_initiall_cell_values.values.shape)

In [58]:
%%capture

initial_values = composedMatrix[..., 0]
cells_cmap = plt.cm.Pastel1

# Set nan's to be displayed as red dots
cells_cmap.set_bad((1,0,0,1))

def do_compose_image(images, index):
    composed_image = images[..., index]
    composed_image[initial_values != 0] = np.nan
    return composed_image

index = 0
fig, ax = plt.subplots(figsize=(15,10))
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plottedImage = ax.imshow(composedMatrix[..., index], cmap=cells_cmap)
stepsCount = composedMatrix.shape[2]

In [59]:
def updateImage(*args):
    global index
    index += 1
    index = index % stepsCount
    plottedImage.set_array(do_compose_image(composedMatrix, index))
    return plottedImage,
    
simulationAnimation = animation.FuncAnimation(fig, updateImage, interval= 100)

HTML(simulationAnimation.to_html5_video())

Iterate over several neighbourhoods, and generate animation for each, to compare them side by side.

Maybe, a neighbourhood object can be defined, and passed as parameter to the model.